Conda `clip2point`

In [13]:
import sys
import pickle
import tqdm 
import glob
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime 

A changer !

In [14]:
# nb vue
nb_view = 12
view_config = "circular"

# Data sur les projections
dir_projection = f"/media/disk1/mpelissi-data/MVTN/{view_config}-{nb_view}/Projections"
# Output
dir_output = f"/media/disk1/mpelissi-data/MVTN/{view_config}-{nb_view}/BVS"
print(f"Input directory: {dir_projection}")
print(f"Output directory: {dir_output}")

Input directory: /media/disk1/mpelissi-data/MVTN/circular-12/Projections
Output directory: /media/disk1/mpelissi-data/MVTN/circular-12/BVS


In [15]:
# Model 3D remeshing iso
dir_remeshing = "/home/mpelissi/Dataset/ModelNet40_remeshing_iso"
paths_iso = glob.glob(os.path.join(dir_remeshing, "*/*/*.obj")); 
paths_projection = glob.glob(os.path.join(dir_projection, "*/*/*cam1_data.npz"))

dir_saillance = "/media/disk1/mpelissi-data/Modelnet40_limper_remeshing_iso_lce"
paths_limper = glob.glob(os.path.join(dir_saillance, "*/*/*.csv")); print(f"Number of saillance files: {len(paths_limper)}")
print(f"🔎​​​ Parmi les {len(paths_iso)} mesh :\nOn a les projections des {nb_view} vues {view_config} de {len(paths_projection)} mesh")

names_limper = [os.path.basename(path).split("_lce")[0] for path in paths_limper]
names_projection = [os.path.basename(path).split("_cam1_data.npz")[0] for path in paths_projection]
names_commun = list(set(names_limper) & set(names_projection))
print(f"Parmi ces {len(paths_projection)} mesh, on a les saillances de {len(names_commun)} mesh")



Number of saillance files: 12288
🔎​​​ Parmi les 12311 mesh :
On a les projections des 12 vues circular de 12301 mesh
Parmi ces 12301 mesh, on a les saillances de 12280 mesh


In [16]:
paths_NAN = []
seuil_NAN = 0.05

In [ ]:
results = []
for path_limper in tqdm.tqdm(paths_limper[:1]):
    name = os.path.basename(path_limper).split('_lce')[0]    
    try:
        #if True:
        # Informations sur le mesh
        cat = path_limper.split('/')[-3]; type = path_limper.split('/')[-2]
        if not os.path.exists(os.path.join(dir_output, cat, type, name+"_bvs.pkl")):
            # Saillance limper associée
            df_saillance_limper = pd.read_csv(path_limper, header=None)
            # colonne Combined_Saliency_Norm
            saillance_limper = np.array([float(s) for s in list(df_saillance_limper[1])[1:]])
            ## les valeurs des saillances sont déjà normalisées entre 0 et 1

            dict_scores = {}
            contain_nan = False
            # Pour chaque pov
            for i in range(0, nb_view): 
                #print(k)
                # data du pov k
                path_npz_cam_i = os.path.join(dir_projection, cat, type, name+"_cam"+str(i+1)+"_data.npz")
                # Load the .npz file and pkl file
                data_cam_i = np.load(path_npz_cam_i)
                # sommets visibles
                sommets_visible = data_cam_i['visible_vertex_idx']#; print(len(sommets_visible))
                #mask_sommets_visible = data_cam_i['visible_vertex_bin']
                cos_angles = data_cam_i['cos_angles']
                # Somme [limper*angle]
                saillance_limper_visible = saillance_limper[sommets_visible]
                if np.isnan(saillance_limper_visible).any() : contain_nan = True
                terme_somme_saillance = np.sum(saillance_limper_visible*cos_angles[sommets_visible])
                if len(np.where(np.isnan(cos_angles[sommets_visible]))[0])>0: print(i+1, name)
                # Surface 3D + 'normalisation' : on divise par la surface 3D totale de l'objet
                terme_surface3d = data_cam_i['surface3D_visible']/data_cam_i['surface3D']
                
                # Sauvegardes des termes
                dict_scores['cam_'+str(i+1)] = {'terme_surface3d': terme_surface3d, 'saillance': terme_somme_saillance}
                
                taux_nan = len(np.where(np.isnan(saillance_limper_visible))[0])/len(saillance_limper_visible)
                if taux_nan > seuil_NAN:
                    contain_nan = True
                    paths_NAN.append((path_limper, taux_nan))
                    break
            
            if not contain_nan:
                # Normalisation du terme de saillance pour les nb_view povs
                max_terme_saillance = np.max([dict_scores['cam_'+str(i+1)]['saillance'] for i in range(0, nb_view)])
                dict_scores = {k: {'terme_surface3d': v['terme_surface3d'], 'saillance': v['saillance'], 
                                    'terme_saillance': v['saillance']/max_terme_saillance} for k, v in dict_scores.items()}

                # Scores = (surface3D + saillance) pour les nb_view povs
                for k, v in dict_scores.items():
                    dict_scores[k]['score'] =  v['terme_surface3d'] + v['terme_saillance']
                    
                # Normalisation des score --> / par la somme des scores
                sum_scores = np.sum([dict_scores[k]['score'] for k in dict_scores.keys()])
                for k, v in dict_scores.items():
                    dict_scores[k]['score_norm'] =  v['score']/sum_scores
                    
                # BVS 
                score_max = np.max([dict_scores[k]['score'] for k in dict_scores.keys()])
                dict_scores['bvs'] = [k for k in dict_scores.keys() if dict_scores[k]['score'] == score_max][0]

                # si plusieurs 
                if len([k for k in dict_scores.keys() if (('cam' in k) and (dict_scores[k]['score'] == score_max))]) > 1: print("Plusieurs pov", name)

                metadata = {
                    "name_limper": path_limper, "categorie": cat, "type": type, "name": name,
                    "bvs" : dict_scores['bvs'], "score_max": score_max, "score_max_norm": dict_scores[dict_scores['bvs']]['score_norm'], 
                    "scores": dict_scores}

                with open(os.path.join(dir_output, cat, type, name+"_bvs.pkl"), "wb") as f: pickle.dump(metadata, f)
                print("Enregistrement de", os.path.join(dir_output, cat, type, name+"_bvs.pkl"))
            
                #Ajout du résultat
                results.append(('ok', path_limper, "RAS", contain_nan))
            else:
                results.append(('nan', path_limper, "Contient des NaN", contain_nan))
        else :
            results.append(("skip", path_limper, "Déjà traité", False))         
    except Exception as e:
        #print(e)
        results.append(("pbl",path_limper, e, contain_nan)) 



100%|██████████| 1/1 [00:00<00:00, 78.61it/s]


In [20]:
with open(os.path.join('/home/mpelissi/MVTN/my_MVTN/data/error', "results_bvs.txt"), "w") as f:
    for p in paths_NAN:
        f.write(f"{p[0]} - Taux de NaN : {p[1]}\n")

In [ ]:
# Write results to file
# List all files in thee current directory
files_in_directory = os.listdir('/home/mpelissi/MVTN/my_MVTN/data/error')
# Filter files containing the word 'error'
nb_error_files = len([file for file in files_in_directory if (('error' in file) and ('bvs' in file))])
if nb_error_files >= 0: 
    file_name = os.path.join('/home/mpelissi/MVTN/my_MVTN/data/error', f"error_bvs_run{nb_error_files+1}_{view_config}_{nb_view}.txt")
    
with open(file_name, "w") as file:
    file.write("Date: {:%Y-%m-%d %H:%M:%S} - Error during saving\n".format(datetime.now()))
    for verdict, name, err, bool_nan in results:
       file.write(f"{verdict}: {name} : {err} -- NAN ? : {bool_nan}\n")